## Structure of the Swatch Class

Swatch implements the Model part of MVVM (Model-View-ViewModel) pattern in .NET MAUI which is then referenced in PaletteViewModel

INotifyPropertyChanged: https://learn.microsoft.com/en-us/dotnet/api/system.componentmodel.inotifypropertychanged?view=net-9.0

This class implements the interface that notifies the UI a property value has changed. In this regards the Swatch Class must have a way to raise an event when a property has changed such as button values and swatch values when the user interacts with the application

OnPropertyChanged() for example

In [ ]:
public class Swatch : INotifyPropertyChanged

The following are private fields used to store the state of the swatch:

In [ ]:
private Color _color;
private bool _isFavorite;
private bool _isLocked;
// private bool _isHovered; -> To use for future hover implementation
private bool _isDeleted;
private Color _previousColor;

The color property have a get and set methods:
+ get => _color means return what is sotre din the private _color field when asked
+ set contains the logic when the user tries to change the Color property (in this context, presses the regenerate button)
    + It checks first to proceed if the swatch is not locked
    + It also checks if the color is different from the current
        + When both conditions are true, the private field is updated with a new color and then OnPropertyChanged() notifies the UI so it can update visually on the swatch

In [ ]:
public Color Color
{
    get => _color;
    set
    {
        if (!_isLocked && _color != value)
        {
            _color = value;
            OnPropertyChanged();
        }
    }
}

The Image properties returns different images based on the state. It does not contain setters, but that's determined by its respective boolean property, thus is readonly. 

In [ ]:
// One of the Image Property
public ImageSource FavoriteImage => ImageSource.FromFile(IsFavorite ? "heart.png" : "unheart.png");

The boolean properties manages the Swatch's state. If the button is clicked, switch to the appropriate response. Example: someone wnats to favorite the Swatch -> switch to Filled Heart image.

In [ ]:
// One of the Boolean Property
public bool IsFavorite
{
    get => _isFavorite;
    set
    {
        if (_isFavorite != value)
        {
            _isFavorite = value;
            OnPropertyChanged();
            OnPropertyChanged(nameof(FavoriteImage));
        }
    }
}

Commands connect UI actions (buttons) to code method. 
https://learn.microsoft.com/en-us/dotnet/api/system.windows.input.icommand?view=net-9.0

In [ ]:
public ICommand ToggleFavoriteCommand { get; }

The Swatch Construcotr initializes the swatch with initial colors and the commands to execute when user interacts with the UI

In [ ]:
public Swatch(Color color)
{
    _color = color;

    ToggleFavoriteCommand = new Command(() => IsFavorite = !IsFavorite);
    ToggleLockCommand = new Command(() => IsLocked = !IsLocked);
    ToggleDeleteCommand = new Command(ToggleDelete);
    ToggleActivateCommand = new Command(() => IsActive = !IsActive);
}

The following method have logic for a Delete command with an Undo option. It will switch buttons states when interacted as well as toggle the buttons visibility when the swatch is clicked.

In [ ]:
// One of the methods
private void ToggleDelete()
{
    if (IsDeleted)
    {
        Color = _previousColor;  
        IsDeleted = false;
        IsActive = false;
        //OnPropertyChanged(nameof(Color));
    }
    else
    {
        _previousColor = Color;  
        Color = Colors.Transparent; 
        IsDeleted = true;
        IsActive = false;
        //OnPropertyChanged(nameof(Color));
    }

    OnPropertyChanged(nameof(Color));
    OnPropertyChanged(nameof(ButtonVisible));
    OnPropertyChanged(nameof(DeleteButtonVisible));
}

Interace: INotifyPropertyChange implementation using OnPropertyChanged method with CallerMemberName attribute:

"Implementing the INotifyPropertyChanged interface when binding data. This interface allows the property of an object to notify a bound control that the property has changed, so that the control can display the updated information."

https://learn.microsoft.com/en-us/dotnet/api/system.runtime.compilerservices.callermembernameattribute?view=net-9.0

[CallerMemberName] automatically fills the parameter with the name of the method OnPropertyChanged.

In [ ]:
public event PropertyChangedEventHandler PropertyChanged;

protected void OnPropertyChanged([CallerMemberName] string name = null) =>
    PropertyChanged?.Invoke(this, new PropertyChangedEventArgs(name));